In [ ]:
CREATE DATABASE TrainSchedules;

In [ ]:
USE TrainSchedules;

In [ ]:
create table TrainTypes (
    ttid int primary key,
    ttname varchar(50),
    ttdesc varchar(255)
);

In [ ]:
create table Trains (
    tid int primary key,
    tname varchar(50),
    ttid int foreign key references TrainTypes(ttid)
);

In [ ]:
create table Routes (
    rid int primary key,
    rname varchar(50) unique,
    tid int foreign key references Trains(tid)
);

In [ ]:
create table Stations (
    sid int primary key,
    sname varchar(50) unique
);

In [ ]:
create table Schedule (
    rid int foreign key references Routes(rid),
    sid int foreign key references Stations(sid),
    atime time,
    dtime time,
    primary key (rid, sid)
);

In [ ]:
CREATE PROCEDURE AddRoute (@Sid INT, @Rid INT, @Atime TIME, @Dtime TIME) AS
BEGIN
    IF EXISTS (SELECT rid, sid FROM Schedule WHERE rid=@Rid and sid=@Sid)
    BEGIN
        -- update the staion from that route with the given arrival and departure times
        UPDATE Scedule SET atime=@Atime, dtime=@Dtime WHERE rid=@Rid and sid=@Sid
    END
    ELSE
    BEGIN
        -- insert station to route
        INSERT INTO Schedule VALUES (@Rid, @Sid, @Atime, @Dtime)
    END
END

In [ ]:
CREATE VIEW PassAllStations AS
    SELECT R.rid AS 'Route ID' FROM Routes R WHERE
        (SELECT COUNT(DISTINCT S.Sid) FROM Schedule S WHERE S.rid = R.rid)
        = 
        (SELECT COUNT(*) FROM Stations) 

In [ ]:
CREATE FUNCTION MoreThanRRoutes (@R INT) 
RETURNS TABLE
AS 
RETURN (
    SELECT S.sname as 'Station Name' FROM Stations S
    WHERE (
        SELECT COUNT(DISTINCT rid) 
        FROM Schedule 
        WHERE S.sid = sid
    ) > @R
)